In [ ]:
import json
import os

from matplotlib import pyplot as plt
import numpy as np

from research.utils.data_access_utils import RDSAccessUtils, S3AccessUtils
from research.weight_estimation.keypoint_utils.optics import pixel2world
import pandas as pd

import plotly.express as px
import seaborn as sns

import sys
sys.path.append('../q1_o2kr2_dataset_annotations')
from thumbnail_selector import get_capture_keys


In [ ]:
def _add_date_hour_us_columns(df):
    """Adds date and hour columns to DataFrame of biomass computations"""
    df.index = list(range(df.shape[0]))
    df = df.sort_values('captured_at').copy(deep=True)
    df.index = pd.to_datetime(df.captured_at)
    dates = df.index.date.astype(str)
    df['date'] = dates
    df['hour'] = df.index.hour
    df['microsecond'] = df.index.microsecond
    return df

def _add_depth(df):
    yaws, pitches, depths = [], [], []
    for idx, row in df.iterrows():
        ann, cm = row.annotation, row.camera_metadata
        try:
            world_keypoints = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
            depth = np.median([x[1] for x in world_keypoints.values()])
            v = world_keypoints['UPPER_LIP'] - world_keypoints['TAIL_NOTCH']
            yaw = np.arctan(v[1] / abs(v[0])) * (180.0 / np.pi)
            pitch = np.arctan(v[2] / abs(v[0])) * (180.0 / np.pi)
        except TypeError as err:
            yaw, pitch, depth = None, None, None
        yaws.append(yaw)
        pitches.append(pitch)
        depths.append(depth)
    df['yaw'] = yaws
    df['pitch'] = pitches
    df['depth'] = depths
    return df

In [ ]:
rds = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
pen_id = 165
start_date, end_date = '2020-10-20', '2020-10-30'
query = """
    select * from prod.biomass_computations
    where pen_id={}
    and captured_at between '{}' and '{}'
    and akpd_score > 0.95;
""".format(pen_id, start_date, end_date)
df = rds.extract_from_database(query)
df = _add_date_hour_columns(df)
df = _add_depth(df)
mask = (df.hour >= 7) & (df.hour <= 15)
df = df[mask].copy(deep=True)

In [ ]:
s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))

In [ ]:
keys = get_capture_keys(165, '2020-10-20', '2020-10-30')

In [ ]:
def get_fov_cutoffs(fov, cm):
    fov = fov * np.pi / 180.0
    field_size_px = 2*cm['focalLengthPixel'] * np.tan(fov / 2.0)
    min_cutoff = (cm['pixelCountWidth'] - field_size_px) / 2.0
    max_cutoff = (cm['pixelCountWidth'] + field_size_px) / 2.0
    return min_cutoff, max_cutoff

def is_preserved(keypoints, min_cutoff, max_cutoff):
    min_x_left = min([item['xFrame'] for item in keypoints['leftCrop']])
    max_x_left = max([item['xFrame'] for item in keypoints['leftCrop']])
    min_x_right = min([item['xFrame'] for item in keypoints['rightCrop']])
    max_x_right = max([item['xFrame'] for item in keypoints['rightCrop']])
    
    if (min_x_left < min_cutoff) or (min_x_right < min_cutoff) or (max_x_left > max_cutoff) or (max_x_right > max_cutoff):
        return False
    return True
        

In [ ]:
fovs = list(np.arange(10, 55, 1))
for fov in fovs:
    min_cutoff, max_cutoff = get_fov_cutoffs(fov, df.camera_metadata.iloc[0])
    is_preserved_list = []
    for idx, row in df.iterrows():
        ann = row.annotation
        is_preserved_list.append(is_preserved(ann, min_cutoff, max_cutoff))

    df['is_preserved_{}'.format(fov)] = is_preserved_list



In [ ]:
pred_weight_means, sample_sizes = [], []
for fov in fovs:
    mask = (df['is_preserved_{}'.format(fov)] == True)
    pred_weight_means.append(df[mask].estimated_weight_g.mean())
    sample_sizes.append(df[mask].shape[0])


In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(fovs, pred_weight_means, label='simulated average weight')
plt.plot(fovs, 1.02 * np.array(pred_weight_means), color='red', linestyle='--', label='+2%')
plt.plot(fovs, 0.98 * np.array(pred_weight_means), color='red', linestyle='--', label='-2%')
plt.xlabel('Field of View (degrees)')
plt.ylabel('Estimated biomass (g)')
plt.title('Empirlcal Optical Samling Bias')
plt.legend()
plt.grid()
plt.show()



In [ ]:
def convert_to_capture_key(image_url):
    return os.path.join(os.path.dirname(image_url.replace('https://aquabyte-crops.s3.eu-west-1.amazonaws.com/', '')), 'capture.json')



In [ ]:
index_vals = []
count = 0
for url in df.left_crop_url.values:
    index_val = keys.index(convert_to_capture_key(url))
    index_vals.append(index_val)
    
    if count % 100 == 0:
        print(count)
    count += 1
    

In [ ]:
df['index_val'] = index_vals

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df[(df.is_preserved_54 == True) & (df.akpd_score > 0.95)].depth.values, bins=20)
plt.hist(df[(df.is_preserved_54 == True) & (df.akpd_score > 0.95) & (df.index_val.apply(lambda x: x % 2 == 0))].depth.values, bins=20)
plt.hist(df[(df.is_preserved_54 == True) & (df.akpd_score > 0.95) & (df.index_val.apply(lambda x: x % 4 == 0))].depth.values, bins=20)
plt.hist(df[(df.is_preserved_54 == True) & (df.akpd_score > 0.95) & (df.index_val.apply(lambda x: x % 8 == 0))].depth.values, bins=20)
plt.hist(df[(df.is_preserved_54 == True) & (df.akpd_score > 0.95) & (df.index_val.apply(lambda x: x % 16 == 0))].depth.values, bins=20)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df[(df.is_preserved_54 == True) & (df.akpd_score > 0.95)].depth.values, bins=20)
plt.hist(df[(df.is_preserved_50 == True) & (df.akpd_score > 0.95)].depth.values, bins=20)
plt.hist(df[(df.is_preserved_46 == True) & (df.akpd_score > 0.95)].depth.values, bins=20)
plt.hist(df[(df.is_preserved_42 == True) & (df.akpd_score > 0.95)].depth.values, bins=20)
plt.grid()
plt.show()

In [ ]:
akpd_mask = df.akpd_score > 0.95
fovs = range(42, 55, 1)
fps_degradations = [1, 2, 4, 8, 16]

sample_sizes = np.zeros((len(fovs), len(fps_degradations)))
for i, fov in enumerate(fovs):
    for j, fps_degradation in enumerate(fps_degradations):
        mask = (df['is_preserved_{}'.format(fov)] == True) & (df.index_val.apply(lambda x: x % fps_degradation == 0)) & akpd_mask
        sample_size = df[mask].shape[0]
        sample_sizes[i, j] = sample_size
        
        
        

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(sample_sizes)
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(fovs, sample_sizes[:, 0], label='simulated sample size')
plt.xlabel('Field of View (degrees)')
plt.ylabel('Sample Size')
plt.title('Sample size vs. FOV')
plt.legend()
plt.grid()
plt.show()



In [ ]:
plt.figure(figsize=(15, 8))
plt.plot([1 / float(x) for x in fps_degradations], sample_sizes[-1, :], label='simulated sample size')
plt.xlabel('FPS')
plt.ylabel('Sample Size')
plt.title('Sample size vs. FPS')
plt.legend()
plt.grid()
plt.show()



<h1> Perform Regression Analysis </h1>

In [ ]:
from collections import defaultdict

akpd_mask = df.akpd_score > 0.95
depth_mask = df.depth < 5.0

fovs = range(42, 55, 1)
fps_degradations = range(1, 16, 1)

baseline_mask = (df['is_preserved_54'] == True) & \
                akpd_mask & \
                depth_mask 
    
baseline_sample_size = df[baseline_mask].shape[0]

data = defaultdict(list)
for i, fov in enumerate(fovs):
    for j, fps_degradation in enumerate(fps_degradations):
        
        mask = (df['is_preserved_{}'.format(fov)] == True) & \
               (df.index_val.apply(lambda x: x % fps_degradation == 0)) & \
               akpd_mask & \
               depth_mask
        
        sample_size = df[mask].shape[0]
        
        data['fov'].append(fov)
        data['fov_squared'].append(fov**2)
        data['fov_fps'].append(fov * 1.0 / fps_degradation)
        data['fps'].append(1.0 / fps_degradation)
        data['sample_size_ratio'].append(sample_size / baseline_sample_size)

In [ ]:
tdf = pd.DataFrame(data)
y = tdf['sample_size_ratio'].values
X = tdf[['fov', 'fov_fps', 'fps']].values

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression().fit(X, y)

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
lr.score(X, y)

In [ ]:
fovs = range(55, 70, 1)
fps_vals = [1, 2, 3, 4, 5]

sample_size_pcts = np.zeros((len(fovs), len(fps_vals)))
for i, fov in enumerate(fovs):
    for j, fps in enumerate(fps_vals):
        X_test = np.array([[fov, fov*fps, fps]])
        sample_size_pct = lr.predict(X_test)[0]
        sample_size_pcts[i, j] = sample_size_pct
        
        
        

In [ ]:
plt.figure(figsize=(20, 10))
sns.heatmap(sample_size_pcts, annot=True, xticklabels=fps_vals, yticklabels=fovs)
plt.show()

In [ ]:
keys[5000]

In [ ]:
3600*2 + 60*21 + 41

In [ ]:
5000/ 8501

In [ ]:
1/1.7

In [ ]:
len(df[(df.is_preserved_54 == True) & (df.akpd_score > 0.95)].depth.values)

In [ ]:
len(df[(df.is_preserved_50 == True) & (df.akpd_score > 0.95)].depth.values)

In [ ]:
yaws, pitches = [], []
for idx, row in df.iterrows():
    ann = row.annotation
    cm = row.camera_metadata
    world_keypoints = pixel2world(ann['leftCrop'], ann['rightCrop'], cm)
    v = world_keypoints['PECTORAL_FIN'] - world_keypoints['ANAL_FIN']
    yaw = np.arctan(v[1] / v[0]) * np.sign(v[0]) * 180.0 / np.pi
    pitch = np.arctan(v[2] / v[0]) * np.sign(v[0]) * 180.0 / np.pi
    yaws.append(yaw)
    pitches.append(pitch)

df['yaw'] = yaws
df['pitch'] = pitches

In [ ]:
plt.hist(df.yaw.abs(), bins=50)
plt.grid()
plt.show()

In [ ]:
df[df.estimated_weight_g > 9000].shape[0] / df.shape[0]

In [ ]:
m = (pred_weight_means[-1] - pred_weight_means[-10]) / (fovs[-1] - fovs[-10])


In [ ]:
12.5 * (1.31 / 2.0)

In [ ]:
plt.figure(figsize=(20, 10))
plt.scatter(fovs, sample_sizes, s=80)
plt.xlabel('Field of View (degrees)')
plt.ylabel('Estimated biomass (g)')
plt.title('Empirlcal Optical Samling Bias')
plt.grid()
plt.show()



In [ ]:
def display_crops(left_image_f, right_image_f, ann, overlay_keypoints=True, show_labels=False):
    fig, axes = plt.subplots(2, 1, figsize=(20, 20))
    left_image = plt.imread(left_image_f)
    right_image = plt.imread(right_image_f)
    axes[0].imshow(left_image)
    axes[1].imshow(right_image)
    
    left_ann, right_ann = ann['leftCrop'], ann['rightCrop']
    left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in left_ann}
    right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in right_ann}
    
    
    if overlay_keypoints:
        for bp, kp in left_keypoints.items():
            axes[0].scatter([kp[0]], [kp[1]], color='red', s=10)
            if show_labels:
                axes[0].annotate(bp, (kp[0], kp[1]), color='red')
        for bp, kp in right_keypoints.items():
            axes[1].scatter([kp[0]], [kp[1]], color='red', s=10)
            if show_labels:
                axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:

s3 = S3AccessUtils('/root/data', json.load(open(os.environ['AWS_CREDENTIALS'])))
row = df[df.estimated_weight_g > 9000].iloc[0]
left_crop_url, right_crop_url = row.left_crop_url, row.right_crop_url
left_crop_f, _, _ = s3.download_from_url(left_crop_url)
right_crop_f, _, _ = s3.download_from_url(right_crop_url)
ann = row.annotation
display_crops(left_crop_f, right_crop_f, ann)

In [ ]:
def compute_depth(ann):
    world_keypoints = [item['']]